## Setup
Installs and imports necessary modules from Github.

Trouble running this? Try restarting the Colab session ("Runtime" --> "Restart Session").

If you're working in your own branch in the Github repo, change the wget commands to point towards that branch.

In [1]:
!mkdir -p modules
!wget -q https://raw.githubusercontent.com/cathat00/NMA_B-T_Project/calebs_branch/modules/network.py -O modules/network.py
!wget -q https://raw.githubusercontent.com/cathat00/NMA_B-T_Project/calebs_branch/modules/tasks.py -O modules/tasks.py
!wget -q https://raw.githubusercontent.com/cathat00/NMA_B-T_Project/calebs_branch/modules/bci.py -O modules/bci.py
!wget -q https://raw.githubusercontent.com/cathat00/NMA_B-T_Project/calebs_branch/modules/plots.py -O modules/plots.py

import numpy as np
import matplotlib.pyplot as plt

import modules.tasks as tasks
from modules.network import RNN
from modules.bci import BCI
from modules.plots import plot_manifold_and_scree

##Train and Evaluate the Model

In [2]:
# ====================
# == Initialization ==
# ====================

np.random.seed(2) # Random seed for this simulation

task = tasks.BasicReachingTask() # The task the RNN will learn
rnn = RNN(N_in=task.ntargets, verbosity=1) # Recurrent Neural Network
bci = BCI(rnn, task.target_max) # Brain computer interface
feedback = np.linalg.pinv(bci.decoder) # Mathematically optimal feedback


# =======================
# == Simulate Learning ==
# =======================
ntrials = 80 # Number of trials to train the RNN for
ntrials_manifold = 50 # Number of trials for manifold computation

# -- Train the RNN
print(f"Training the RNN for {ntrials} trials...")
rnn.relearn(
    ntrials=ntrials,
    ext=task.stimuli,
    ntstart=task.stim_length,
    decoder=bci.decoder,
    feedback=feedback,
    target=task.targets,
    delta=20.,
)

# -- Compute Manifold
print(f"Computing manifold over {ntrials_manifold} trials...")
manifold_data = rnn.get_manifold(task.stimuli, task.stim_length)
manifold_activity = manifold_data['proj']
eigenvalues = manifold_data['eigenvals']

# -- Train the BCI
# TODO: What's the need for :10 / AKA "reduced_dim"?
bci.train(
    manifold_data["proj_reshaped"][:, :, :10],
    task.targets[:, task.stim_length:, :],
    manifold_data["order"]
)


# =======================
# == Evaluate Learning ==
# =======================
# -- Calculate MSE
# (MSE must be computed retrospectively using the trained BCI)
# ... TODO
# -- Calculate Manifold Surface Area / Perturbation Sensitity
# ... TODO


# =======================
# == Visualize Results ==
# =======================
# -- Plot MSE
# ...TODO
# -- Plot Manifold and Scree Plots
plot_manifold_and_scree(manifold_activity, eigenvalues)
# -- TODO (Other Plots?)

Training the RNN for 80 trials...


100%|██████████| 80/80 [06:26<00:00,  4.83s/it]


Computing manifold over 50 trials...


100%|██████████| 50/50 [00:03<00:00, 13.18it/s]


NameError: name 'manifold_out' is not defined